In [3]:
import numpy as np
import tensorflow as tf
import helper #formatting the data and generating random sequence data

In [4]:
tf.reset_default_graph()
sess=tf.InteractiveSession()

In [5]:
tf.__version__

'1.0.1'

### Padding
Every Input is of diffrent length["How r u","this the way i like it"].
Before you you feed this into the network you need to solve this
the way we can solve this is through "Padding".
Nothing but adding zeros at the end.


In [6]:
PAD=0
EOD=1
vocab_size=10#Max Length for the input data
#we gonna covert the input sentence into a embedding vector then feed it into
#the network.
input_embedding_size=20#no of charachters

encoder_hidden_units=20
decoder_hidden_units=encoder_hidden_units*2
#in orginal paper encoder=hiddenunits

In [7]:
#placeholders
#3 place holders for Inputs,Decoder,Target
encoder_inputs=tf.placeholder(shape=(None,None),dtype=tf.int32,name="encoder_inputs")
encoder_inputs_length=tf.placeholder(shape=(None,None),dtype=tf.int32,name="encoder_inputs_length")
decoder_target=tf.placeholder(shape=(None,None),dtype=tf.int32,name="decoder_target")

In [8]:
#embeddings
#tf.random_uniform generates a random matrix having size of voacb size*input_embedding_size..with random values between -1 to 1
#it has a normal distrubution between -1 to 1
embeddings=tf.Variable(tf.random_uniform([vocab_size,input_embedding_size],-1,1),dtype=tf.float32)

In [9]:
#we need to keep our inputs into the embedding matrix which we created and then feed that matrix into the encoder
encoder_inputs_embeded=tf.nn.embedding_lookup(embeddings,encoder_inputs)

In [10]:
#define encoder
encoder_cell = tf.contrib.rnn.BasicLSTMCell(encoder_hidden_units)




In [13]:
((encoder_fw_outputs,
  encoder_bw_outputs),
 (encoder_fw_final_state,
  encoder_bw_final_state)) = (
    tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                    cell_bw=encoder_cell,
                                    inputs=encoder_inputs_embeded,
                                    sequence_length=encoder_inputs_length,
                                    dtype=tf.int32, time_major=True)
    )

ValueError: sequence_length must be a vector of length batch_size, but saw shape: (?, ?)

In [ ]:

#Concatenates tensors along one dimension.
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

#letters h and c are commonly used to denote "output value" and "cell state". 
#http://colah.github.io/posts/2015-08-Understanding-LSTMs/ 
#Those tensors represent combined internal state of the cell, and should be passed together. 

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

#TF Tuple used by LSTM Cells for state_size, zero_state, and output state.
encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h
)

In [14]:
#decoder
encoder_cell = tf.contrib.rnn.BasicLSTMCell(decoder_hidden_units)
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))
decoder_lengths = encoder_inputs_length + 3

In [15]:
#manually specifying since we are going to implement attention details for the decoder in a sec
#weights
W = tf.Variable(tf.random_uniform([decoder_hidden_units, vocab_size], -1, 1), dtype=tf.float32)
#bias
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [16]:
#create padded inputs for the decoder from the word embeddings

#were telling the program to test a condition, and trigger an error if the condition is false.
assert EOS == 1 and PAD == 0

eos_time_slice = tf.ones([batch_size], dtype=tf.int32, name='EOS')
pad_time_slice = tf.zeros([batch_size], dtype=tf.int32, name='PAD')

#retrieves rows of the params tensor. The behavior is similar to using indexing with arrays in numpy
eos_step_embedded = tf.nn.embedding_lookup(embeddings, eos_time_slice)
pad_step_embedded = tf.nn.embedding_lookup(embeddings, pad_time_slice)

NameError: name 'EOS' is not defined

In [17]:

#manually specifying loop function through time - to get initial cell state and input to RNN
#normally we'd just use dynamic_rnn, but lets get detailed here with raw_rnn

#we define and return these values, no operations occur here
def loop_fn_initial():
    initial_elements_finished = (0 >= decoder_lengths)  # all False at the initial step
    #end of sentence
    initial_input = eos_step_embedded
    #last time steps cell state
    initial_cell_state = encoder_final_state
    #none
    initial_cell_output = None
    #none
    initial_loop_state = None  # we don't need to pass any additional information
    return (initial_elements_finished,
            initial_input,
            initial_cell_state,
            initial_cell_output,
            initial_loop_state)

In [18]:
#attention mechanism --choose which previously generated token to pass as input in the next timestep
def loop_fn_transition(time, previous_output, previous_state, previous_loop_state):

    
    def get_next_input():
        #dot product between previous ouput and weights, then + biases
        output_logits = tf.add(tf.matmul(previous_output, W), b)
        #Logits simply means that the function operates on the unscaled output of 
        #earlier layers and that the relative scale to understand the units is linear. 
        #It means, in particular, the sum of the inputs may not equal 1, that the values are not probabilities 
        #(you might have an input of 5).
        #prediction value at current time step
        
        #Returns the index with the largest value across axes of a tensor.
        prediction = tf.argmax(output_logits, axis=1)
        #embed prediction for the next input
        next_input = tf.nn.embedding_lookup(embeddings, prediction)
        return next_input
    
    
    elements_finished = (time >= decoder_lengths) # this operation produces boolean tensor of [batch_size]
                                                  # defining if corresponding sequence has ended

    
    
    #Computes the "logical and" of elements across dimensions of a tensor.
    finished = tf.reduce_all(elements_finished) # -> boolean scalar
    #Return either fn1() or fn2() based on the boolean predicate pred.
    input = tf.cond(finished, lambda: pad_step_embedded, get_next_input)
    
    #set previous to current
    state = previous_state
    output = previous_output
    loop_state = None

    return (elements_finished, 
            input,
            state,
            output,
            loop_state)